In [8]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from algorithms.autoencoder import autoencoder as ae
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder as onehot
from imblearn.over_sampling import RandomOverSampler

path = "/media/jahan/solo/"
os.chdir(path)

# prep data
def prep(csv):   
    labels = pd.read_csv(csv)
    labels = labels[labels["Frontal/Lateral"] == "Frontal"]
    normal = labels[labels["No Finding"] == 1]
    normal = normal.Path
    abnormal = labels[labels["No Finding"] != 1]
    abnormal = abnormal.Path
    print(len(normal),len(abnormal))

    x_a = pd.concat([normal,abnormal]).values
    indexes_switch = np.random.choice(len(x_a), int(len(x_a)/2), replace=False)
    x_b = pd.concat([normal.sample(frac=1),abnormal.sample(frac=1)]).values
    x_c = pd.concat([abnormal.sample(n=len(normal)),normal.sample(n=len(abnormal),replace=True)]).values

    temp_b = x_b[indexes_switch]
    temp_c = x_c[indexes_switch]
    x_b[indexes_switch] = temp_c
    x_c[indexes_switch] = temp_b
    
    x = np.stack((x_a,x_b,x_c), axis=-1)
    
    y = np.full((len(x_a),2),[0,1])
    y[indexes_switch] = np.full((len(indexes_switch),2),[1,0])
    
    sm = RandomOverSampler(random_state=42)
    X_res, y_res = sm.fit_resample(x, y)
    
    oh = onehot()
    y = oh.fit_transform(y_res)
    
    return X_res[:,0],X_res[:,1],X_res[:,2], y.toarray()

x_a,x_b,x_c, y = prep("CheXpert-v1.0-small/train.csv")

print(y)

config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


save_path = "/home/jahan/Documents/CXR_CNN/output/triple"



encoder = ae(a_paths=x_a, b_paths=x_b, c_paths=x_c, labels=y, save_path=save_path)
#encoder.build_dataset(batch_size=32)
encoder.build(load="/triple.ckpt")

16974 174053


/home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_140 (Conv2D)             (None, 256, 256, 16) 416         input_22[0][0]                   
                                                                 input_23[0][0]                   
                                                                 input_24[0][0]                   
__________________________________________________________________________________________________
batch_normalization_140 (BatchN (None, 256, 256, 16) 64          conv2d_140[0][0]                 
                                

OSError: Unable to open file (unable to open file: name = '/home/jahan/Documents/CXR_CNN/output/triple/triple.ckpt', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [2]:
encoder.train(10)

W0815 20:38:17.257627 140326057015104 deprecation.py:323] From /home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
799999/800000 [============================>.] - ETA: 0s - loss: 0.1392 - binary_crossentropy: 0.1390 - auc: 0.9881 - binary_accuracy: 0.9431
Epoch 00001: saving model to /home/jahan/Documents/CXR_CNN/output/tripletriple.ckpt
800000/800000 [==============================] - 119504s 149ms/step - loss: 0.1392 - binary_crossentropy: 0.1390 - auc: 0.9881 - binary_accuracy: 0.9431
Epoch 2/10
799999/800000 [============================>.] - ETA: 0s - loss: 0.0515 - binary_crossentropy: 0.0514 - auc: 0.9981 - binary_accuracy: 0.9798
Epoch 00002: saving model to /home/jahan/Documents/CXR_CNN/output/tripletriple.ckpt
800000/800000 [==============================] - 118021s 148ms/step - loss: 0.0515 - binary_crossentropy: 0.0514 - auc: 0.9981 - binary_accuracy: 0.9798
Epoch 3/10
799999/800000 [============================>.] - ETA: 0s - loss: 0.0377 - binary_crossentropy: 0.0376 - auc: 0.9988 - binary_accuracy: 0.9857
Epoch 00003: saving model to /home/jahan/Documents/CXR_CNN/output/t

KeyboardInterrupt: 

In [ ]:
x_a,x_b,x_v, y = prep("CheXpert-v1.0-small/valid.csv")

